<h1>IBM Capstone Project</h1>

<h2>Comparison between NYC, USA and Toronto, CA</h2>

## Introduction: Business Problem <br>
In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening a new restaurant either in NYC or Toronto based on the recommendation.
We'll be analysing different venues available at both the cities, which will helps to understand types of venues in these cities and which venue is most famous. <br>
We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* type of restaurant present in the neighborhood
* popularity of the venue


Following data sources will be needed to extract/generate the required information:
* Co-ordinates data of NYC, USA as well as Toronto, CA.
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**


## Methodology ¶

<p>
In this project we will direct our efforts on detecting areas of NYC and Toronto to figure out the most common venues in both these cities.

In first step we have collected the required data which consist of borough within these cities and their respective co-ordinates.

Second step in our analysis will explore the various venues around the location of these cities , we will use maps to identify a few promising areas focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create clusters of locations that meet some basic requirements discussed in the interest of the stakeholders. We will present map of all such locations but also create clusters (using k-means clustering) of those locations to identify what are the venues which attracts more audience here and which city is better than other for what kind of investment.</p>

<b>Download the required libraries</b>

In [296]:
import pandas as pd # library for data analsysis

print('Libraries imported.')

Libraries imported.


In [297]:
#import beautifulsoup
from bs4 import BeautifulSoup
#import urllib package
import urllib
import urllib.request

In [298]:
#open wiki page with BeautifulSoup
beautifulSoup = BeautifulSoup(urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read(),"lxml")
table = beautifulSoup.find("table",{"class":"wikitable sortable"}) # Grab the first table
new_table = pd.DataFrame(columns=range(0,3), index = [0]) # I know the size 
df_toro = pd.read_html(str(table))[0]


In [299]:
#merge neighborhood based on postcode
df_toro = df_toro.groupby('Postcode').agg({'Borough':'first','Neighbourhood': ', '.join}).reset_index()
#Remove records where Borough is Not Assigned
df_toro.drop(df_toro[df_toro.Borough == 'Not assigned'].index, inplace=True)
#replace value of Not Assigned Neighbourhood with value of Borough
df_toro.loc[df_toro['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df_toro['Borough']

df_toronto = df_toro.rename(index=str, columns={"Postcode": "PostalCode"})

df_toronto.head()

,PostalCode,Borough,Neighbourhood
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [300]:
df_toronto.shape

(103, 3)

In [301]:
#get the NYC data
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [302]:
#load NYC data

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [303]:
# define the dataframe columns
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
df_nyc = pd.DataFrame(columns=column_names)

In [304]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    df_nyc = df_nyc.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    

In [305]:
#Data with lattitutes and longitudes NYC
df_nyc.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [306]:
#skipped using geocoder package, response is not reliable to retrive the data
import io
import requests

url="https://cocl.us/Geospatial_data"
s=requests.get(url).content
ll_df=pd.read_csv(io.StringIO(s.decode('utf-8')))
new_ll_df = ll_df.rename(index=str, columns={"Postal Code": "PostalCode"})


<b>Toronto -merged data with zipcodes latitude and longitude</b>

In [307]:
merged_df_toronto = pd.merge(df_toronto, new_ll_df, on='PostalCode')
#Data with lattitutes and longitudes Toronto
merged_df_toronto.head()


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [308]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

<b>Get the Toronto and NYC latitide and longitue</b>

In [309]:
address_toronto = 'Toronto, CA'
address_nyc = 'New York City, NY'
geolocator = Nominatim(user_agent="ny_explorer")
t_location = geolocator.geocode(address_toronto)
t_latitude = t_location.latitude
n_location = geolocator.geocode(address_nyc)
n_latitude = n_location.latitude

t_longitude = t_location.longitude
n_longitude = n_location.longitude
print('The geograpical coordinate of Toronto, CA are {}, {}.'.format(t_latitude, t_longitude))
print('The geograpical coordinate of New York City, NY are {}, {}.'.format(n_latitude, n_longitude))

The geograpical coordinate of Toronto, CA are 43.653963, -79.387207.
The geograpical coordinate of New York City, NY are 40.7127281, -74.0060152.


#### Create a map of Toronto, CA with neighborhoods superimposed on top.

In [310]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[t_latitude, t_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(merged_df_toronto['Latitude'], merged_df_toronto['Longitude'], merged_df_toronto['Borough'], merged_df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### create map for nyc n

In [311]:
# create map of New York using latitude and longitude values
map_nyc = folium.Map(location=[n_latitude, n_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_nyc['Latitude'], df_nyc['Longitude'], df_nyc['Borough'], df_nyc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyc)  
    
map_nyc

### define the foursquare credentials

In [312]:
CLIENT_ID = 'SOSACUHDMK3YVDOIO3LGE0WLNBSQDHORML2LLXULP50HCJFV' # your Foursquare ID
CLIENT_SECRET = 'T1UPC5H5B0COUBVPMMKSLLSL0EV5YQJTLQ5XHWBOS0FO21RO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SOSACUHDMK3YVDOIO3LGE0WLNBSQDHORML2LLXULP50HCJFV
CLIENT_SECRET:T1UPC5H5B0COUBVPMMKSLLSL0EV5YQJTLQ5XHWBOS0FO21RO


#### consider only those Boroughs which has word Toronto

In [313]:
toronto_borough_data = merged_df_toronto[merged_df_toronto['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_borough_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [314]:
### consider records for Manhattan Borough obly

In [315]:
# manhattan
manhattan_data = df_nyc[df_nyc['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [316]:
# create map of boroughs having word as Toronto using latitude and longitude values
map_toronto_borough = folium.Map(location=[t_latitude, t_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_borough_data['Latitude'], toronto_borough_data['Longitude'], toronto_borough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_borough)  
    
map_toronto_borough

In [317]:
# create map of boroughs having word as Manhattan using latitude and longitude values
map_manhattan_borough = folium.Map(location=[n_latitude, n_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan_borough)  
    
map_manhattan_borough

In [318]:
toronto_borough_data.loc[0, 'Neighbourhood']

'The Beaches'

#### Get the latitute and longitude of the first location

In [319]:
neighbourhood_latitude = toronto_borough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = toronto_borough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = toronto_borough_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [320]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL





'https://api.foursquare.com/v2/venues/explore?&client_id=SOSACUHDMK3YVDOIO3LGE0WLNBSQDHORML2LLXULP50HCJFV&client_secret=T1UPC5H5B0COUBVPMMKSLLSL0EV5YQJTLQ5XHWBOS0FO21RO&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [321]:
results = requests.get(url).json()

In [322]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [323]:
from pandas.io.json import json_normalize

In [324]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,St-Denis Studios Inc.,Music Venue,43.675031,-79.288022
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [325]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


#### Explore the neighbourhoods

In [326]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Gather venues around Toronto

In [327]:
toronto_venues = getNearbyVenues(names=toronto_borough_data['Neighbourhood'],
                                   latitudes=toronto_borough_data['Latitude'],
                                   longitudes=toronto_borough_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [328]:
print(toronto_venues.shape)
toronto_venues.head()

(1700, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,St-Denis Studios Inc.,43.675031,-79.288022,Music Venue
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


#### count of Venues returned for each neighbourhood

In [329]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton, Exhibition Place, Parkdale Village",19,19,19,19,19,19
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",44,44,44,44,44,44
Central Bay Street,88,88,88,88,88,88
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [330]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


#### Analyzing neighbourhoods

In [331]:
#analyze each neighborhood
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [332]:
toronto_onehot.shape

(1700, 237)

In [333]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,...,0.00,0.00000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.00000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,...,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.022727,...,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011364,...,0.00,0.00000,0.000000,0.00,0.011364,0.000000,0.000000,0.011364,0.000000,0.011364
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.01,0.01000,0.000000,0.00,0.060000,0.000000,0.030000,0.010000,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.011364,0.011364,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011364,...,0.00,0.00000,0.000000,0.00,0.000000,0.011364,0.011364,0.000000,0.011364,0.011364


In [334]:
toronto_grouped.shape

(38, 237)

In [335]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2    Restaurant  0.04
3          Café  0.04
4      Beer Bar  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.11
1     Coffee Shop  0.11
2            Café  0.11
3    Climbing Gym  0.05
4   Grocery Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2          Restaurant  0.06
3             Brewery  0.06
4         Pizza Place  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.20
1    Airport Lounge  0.13
2  Airport Termin

### Manhattan venues

In [336]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [337]:
manhattan_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,43,43,43,43,43,43
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,41,41,41,41,41,41
East Village,100,100,100,100,100,100


In [338]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [339]:
import numpy as np

#### display top ten venues of the neighbourhood Toronto

In [340]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Gym,Bakery,Bar,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Café,Seafood Restaurant,Cheese Shop,Farmers Market,Steakhouse,Beer Bar,Bakery
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Grocery Store,Climbing Gym,Burrito Place,Stadium,Bar,Restaurant,Caribbean Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Recording Studio
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Airport Lounge,Plane,Boat or Ferry,Sculpture Garden,Harbor / Marina,Boutique,Airport Gate,Airport Food Court


### NYC top venues

In [341]:
manhattan_venues.groupby('Neighbourhood').count()

#analyze each neighborhood
# one hot encoding
nyc_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nyc_onehot['Neighbourhood'] = manhattan_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [nyc_onehot.columns[-1]] + list(nyc_onehot.columns[:-1])
nyc_onehot = nyc_onehot[fixed_columns]

nyc_onehot.head()

#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
nyc_grouped = nyc_onehot.groupby('Neighbourhood').mean().reset_index()
nyc_grouped

nyc_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
nyc_neighborhoods_venues_sorted['Neighbourhood'] = nyc_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    nyc_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyc_grouped.iloc[ind, :], num_top_venues)

nyc_neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Gym,Wine Shop,Italian Restaurant,Clothing Store,Plaza,Memorial Site,Burger Joint
1,Carnegie Hill,Pizza Place,Coffee Shop,Café,Cosmetics Shop,Grocery Store,Bar,Spa,French Restaurant,Japanese Restaurant,Yoga Studio
2,Central Harlem,African Restaurant,Public Art,Cosmetics Shop,French Restaurant,Chinese Restaurant,Seafood Restaurant,Gym / Fitness Center,American Restaurant,Park,Southern / Soul Food Restaurant
3,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,Bakery,American Restaurant,Hotel,Seafood Restaurant,Theater,Cupcake Shop
4,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Bubble Tea Shop,Noodle House,Bar,Dumpling Restaurant,Ice Cream Shop,Hotpot Restaurant


<h1>Clustering neighbourhoods Toronto, CA</h1>

In [342]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [343]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 4, 0, 4, 4, 4, 4, 4], dtype=int32)

In [344]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_borough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Music Venue,Health Food Store,Pub,Neighborhood,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,4,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Furniture / Home Store,Italian Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,4,Park,Sandwich Place,Pet Store,Ice Cream Shop,Burger Joint,Liquor Store,Light Rail Station,Burrito Place,Fast Food Restaurant,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,4,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Cheese Shop,Fish Market,Latin American Restaurant,Coworking Space
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Construction & Landscaping,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [345]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<b> Clustering neighbourhoods NYC, USA</b>

In [346]:
nyc_grouped_clustering = nyc_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_nyc = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_nyc.labels_[0:10] 



# add clustering labels
nyc_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans_nyc.labels_)

nyc_merged = manhattan_data

# merge nyc_grouped with manhattan_data to add latitude/longitude for each neighborhood
nyc_merged = nyc_merged.join(nyc_neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

nyc_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Discount Store,Coffee Shop,Yoga Studio,Seafood Restaurant,Gym,Tennis Stadium,Big Box Store,Supplement Shop,Steakhouse,Shoe Store
1,Manhattan,Chinatown,40.715618,-73.994279,4,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Bubble Tea Shop,Noodle House,Bar,Dumpling Restaurant,Ice Cream Shop,Hotpot Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Café,Bakery,Mobile Phone Shop,Grocery Store,Pizza Place,Chinese Restaurant,Latin American Restaurant,Mexican Restaurant,Tapas Restaurant,Shoe Store
3,Manhattan,Inwood,40.867684,-73.921210,4,Lounge,Mexican Restaurant,Café,Pizza Place,Wine Bar,Deli / Bodega,Park,American Restaurant,Frozen Yogurt Shop,Chinese Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Mexican Restaurant,Café,Coffee Shop,Pizza Place,Yoga Studio,Sushi Restaurant,Caribbean Restaurant,Chinese Restaurant,School,Bakery


In [347]:
#cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,0,Park,Construction & Landscaping,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
23,Central Toronto,0,Trail,Sushi Restaurant,Jewelry Store,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
27,Downtown Toronto,0,Airport Service,Airport Terminal,Airport Lounge,Plane,Boat or Ferry,Sculpture Garden,Harbor / Marina,Boutique,Airport Gate,Airport Food Court


In [348]:
#cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Garden,Filipino Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [349]:
#cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,2,Playground,Tennis Court,Trail,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant
10,Downtown Toronto,2,Park,Playground,Trail,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [350]:
#cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,3,Music Venue,Health Food Store,Pub,Neighborhood,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [351]:
#cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,4,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Furniture / Home Store,Italian Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
2,East Toronto,4,Park,Sandwich Place,Pet Store,Ice Cream Shop,Burger Joint,Liquor Store,Light Rail Station,Burrito Place,Fast Food Restaurant,Fish & Chips Shop
3,East Toronto,4,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Cheese Shop,Fish Market,Latin American Restaurant,Coworking Space
5,Central Toronto,4,Gym,Food & Drink Shop,Sandwich Place,Park,Breakfast Spot,Clothing Store,Hotel,Discount Store,Dog Run,Doner Restaurant
6,Central Toronto,4,Clothing Store,Coffee Shop,Yoga Studio,Fast Food Restaurant,Pet Store,Chinese Restaurant,Rental Car Location,Dessert Shop,Salon / Barbershop,Diner
7,Central Toronto,4,Sandwich Place,Pizza Place,Dessert Shop,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Gourmet Shop,Dance Studio,Seafood Restaurant
9,Central Toronto,4,Coffee Shop,Pub,Pizza Place,Sushi Restaurant,Bagel Shop,Fried Chicken Joint,Sports Bar,American Restaurant,Convenience Store,Light Rail Station
11,Downtown Toronto,4,Coffee Shop,Pub,Pizza Place,Restaurant,Café,Market,Italian Restaurant,Bakery,Playground,Liquor Store
12,Downtown Toronto,4,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Nightclub,Bubble Tea Shop,Men's Store,Café,Mediterranean Restaurant
13,Downtown Toronto,4,Coffee Shop,Park,Pub,Café,Bakery,Theater,Mexican Restaurant,Breakfast Spot,Brewery,Italian Restaurant


<b>NYC Clusters</b>

In [352]:
#cluster 1
nyc_merged.loc[nyc_merged['Cluster Labels'] == 0, nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Manhattanville,Coffee Shop,Mexican Restaurant,Italian Restaurant,Park,Chinese Restaurant,Seafood Restaurant,Dumpling Restaurant,Bar,Bus Station,Burger Joint
15,Midtown,Hotel,Clothing Store,Steakhouse,Theater,Cocktail Bar,Food Truck,Spa,American Restaurant,Coffee Shop,Bookstore
26,Morningside Heights,Coffee Shop,American Restaurant,Park,Bookstore,Café,Food Truck,Deli / Bodega,Burger Joint,New American Restaurant,Tennis Court
28,Battery Park City,Park,Coffee Shop,Hotel,Gym,Wine Shop,Italian Restaurant,Clothing Store,Plaza,Memorial Site,Burger Joint
29,Financial District,Coffee Shop,Gym,Wine Shop,Hotel,Steakhouse,Pizza Place,Café,Bar,American Restaurant,Park


In [353]:
#cluster 2
nyc_merged.loc[nyc_merged['Cluster Labels'] == 0, nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Manhattanville,Coffee Shop,Mexican Restaurant,Italian Restaurant,Park,Chinese Restaurant,Seafood Restaurant,Dumpling Restaurant,Bar,Bus Station,Burger Joint
15,Midtown,Hotel,Clothing Store,Steakhouse,Theater,Cocktail Bar,Food Truck,Spa,American Restaurant,Coffee Shop,Bookstore
26,Morningside Heights,Coffee Shop,American Restaurant,Park,Bookstore,Café,Food Truck,Deli / Bodega,Burger Joint,New American Restaurant,Tennis Court
28,Battery Park City,Park,Coffee Shop,Hotel,Gym,Wine Shop,Italian Restaurant,Clothing Store,Plaza,Memorial Site,Burger Joint
29,Financial District,Coffee Shop,Gym,Wine Shop,Hotel,Steakhouse,Pizza Place,Café,Bar,American Restaurant,Park


In [354]:
#cluster 3
nyc_merged.loc[nyc_merged['Cluster Labels'] == 2, nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Italian Restaurant,Exhibit,Coffee Shop,Bakery,Gym / Fitness Center,Art Gallery,Juice Bar,Hotel,Spa,Cocktail Bar
9,Yorkville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Lenox Hill,Coffee Shop,Sushi Restaurant,Italian Restaurant,Pizza Place,Gym / Fitness Center,Burger Joint,Gym,Sporting Goods Shop,Salad Place,Turkish Restaurant
12,Upper West Side,Italian Restaurant,Bar,Wine Bar,Coffee Shop,Mediterranean Restaurant,Vegetarian / Vegan Restaurant,Burger Joint,Indian Restaurant,Bakery,Bookstore
16,Murray Hill,Coffee Shop,Sandwich Place,Hotel,Japanese Restaurant,Bar,French Restaurant,Italian Restaurant,Gym,Bagel Shop,Bakery
17,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,Bakery,American Restaurant,Hotel,Seafood Restaurant,Theater,Cupcake Shop
18,Greenwich Village,Italian Restaurant,Sushi Restaurant,French Restaurant,Clothing Store,Seafood Restaurant,Indian Restaurant,Café,Gourmet Shop,Cosmetics Shop,Bakery
19,East Village,Bar,Wine Bar,Chinese Restaurant,Ice Cream Shop,Mexican Restaurant,Pizza Place,Ramen Restaurant,Coffee Shop,Cocktail Bar,Vegetarian / Vegan Restaurant
21,Tribeca,Park,Italian Restaurant,Café,Spa,American Restaurant,Boutique,Wine Shop,Wine Bar,Gym,Coffee Shop
22,Little Italy,Bakery,Sandwich Place,Mediterranean Restaurant,Clothing Store,Italian Restaurant,Ice Cream Shop,Salon / Barbershop,Seafood Restaurant,Café,Malay Restaurant


In [355]:
#cluster 4
nyc_merged.loc[nyc_merged['Cluster Labels'] == 3, nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Bar,Park,Playground,German Restaurant,Basketball Court,Baseball Field,Fountain,Harbor / Marina,Cocktail Bar,Coffee Shop


In [356]:
#cluster 5

nyc_merged.loc[nyc_merged['Cluster Labels'] == 4, nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Discount Store,Coffee Shop,Yoga Studio,Seafood Restaurant,Gym,Tennis Stadium,Big Box Store,Supplement Shop,Steakhouse,Shoe Store
1,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Bubble Tea Shop,Noodle House,Bar,Dumpling Restaurant,Ice Cream Shop,Hotpot Restaurant
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Grocery Store,Pizza Place,Chinese Restaurant,Latin American Restaurant,Mexican Restaurant,Tapas Restaurant,Shoe Store
3,Inwood,Lounge,Mexican Restaurant,Café,Pizza Place,Wine Bar,Deli / Bodega,Park,American Restaurant,Frozen Yogurt Shop,Chinese Restaurant
4,Hamilton Heights,Mexican Restaurant,Café,Coffee Shop,Pizza Place,Yoga Studio,Sushi Restaurant,Caribbean Restaurant,Chinese Restaurant,School,Bakery
6,Central Harlem,African Restaurant,Public Art,Cosmetics Shop,French Restaurant,Chinese Restaurant,Seafood Restaurant,Gym / Fitness Center,American Restaurant,Park,Southern / Soul Food Restaurant
7,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Deli / Bodega,Thai Restaurant,Convenience Store,Café,Taco Place,Street Art,Steakhouse
11,Roosevelt Island,Sandwich Place,Deli / Bodega,Coffee Shop,Dog Run,Café,Farmers Market,Supermarket,Metro Station,Outdoors & Recreation,Dry Cleaner
20,Lower East Side,Café,Chinese Restaurant,Coffee Shop,Ramen Restaurant,Japanese Restaurant,Cocktail Bar,Shoe Store,Bakery,Art Gallery,Sandwich Place
25,Manhattan Valley,Indian Restaurant,Coffee Shop,Pizza Place,Yoga Studio,Playground,Spa,Café,Mexican Restaurant,Deli / Bodega,Bar


### -----------------------------------------------------------------------------------------------------------------------------

<b>Analysis</b>
<p>
Here we'll go through the basic analysis done in the above code base using the data available.
As we had data of NYC and Toronto borroughs and their co-ordinates details, we used Foursquare location service to get the details about the various venues around these cities.
To narrow down the scope we considered only those locations which are pasrt of Manhattan Borough, NYC and in case is of Toronoto we considered only bourough having word Toronto in their name.
After reducing the location data to certain bouroughs we generate the information about the various types of venues around them. 
Using clustering technique we clustered the location in different clusters for both the cities.
Based on these clusters we analyse and understand that which type of venue is more popular in the locations considered.
    
</p>

<b>Results and Discussion</b>

<p>
 Although our analysis based on clustering shows that both the cities have many popular venues and have good amount of potential for an investor to start a new flavor in the city.
 From our analysis it is clear that though Toronto has many places one may interests in but from all the data clusters it is observed that it has places more popular like parks, gardens or a music venue. <br>
 On contrary NYC is pro-foodie and we can clearly see that it has number of restaurants as most popular venues and definitely interests someone who wants to invest in food/restaurant business.
 <br>
 Among all it is observed that Mexican Food is really popular around Manhattan,NYC. Next to that is Italian Food which has its own impression on NYC along with Pizza places.
</p>

<b>Conclusion</b>

Purpose of this project was to compare Toronto, CA and NYC, USA and two figure what characteristics these cities have and which venues can be point of an interest for an investor.<br>
Based on the analysis and discussion it is very much clear that NYC has more potential for an investment for food/restaurant business.
Again, it depends on an investor what choice his makes but if anyone who's interested to ivest in opening a restaurant in NYC may consider the popular cuisines line Mexican or Italian. Again it varies type of investment as it gives an option for an up-scale or an average restaurant.